In [2]:
import pandas as pd
from pipeline_executor import PipelineExecutor
from nlp_analysis.NER import extract_entities
topic = "'quantumcomputing'AND'research'"


    pipeline_executor = PipelineExecutor()
    quantum_df = pipeline_executor.execute(query=topic, max_articles=20, overwrite=True)

Getting news article info: 100%|██████████| 59/59 [00:27<00:00,  2.17it/s]


In [23]:
pd.read_csv(f"data/clean/'quantumcomputing'AND'research'_20.csv")

,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body
0,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,['Elliot Williams'],2015-09-29 00:00:00,Imagine a world where the most widely-used cry...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
1,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,['Elliot Williams'],2015-09-29 00:00:00,If you take the development of serious quantum...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
2,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,['Elliot Williams'],2015-09-29 00:00:00,"All is not doom and gloom, however. There are ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
3,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,['Elliot Williams'],2015-09-29 00:00:00,"Strong symmetric ciphers, algorithms that use ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
4,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,['Elliot Williams'],2015-09-29 00:00:00,But let’s just reiterate that the hollowing-ou...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
...,...,...,...,...,...,...,...,...,...,...
647,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"['Kevin Krewell', 'Amy Danise', 'Contributor G...",2022-11-17 00:00:00,With the development of quantum computing tech...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
648,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"['Kevin Krewell', 'Amy Danise', 'Contributor G...",2022-11-17 00:00:00,IBM Fellow and VP of Quantum Jay Gambetta ende...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
649,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"['Kevin Krewell', 'Amy Danise', 'Contributor G...",2022-11-17 00:00:00,Even though the concept of a quantum computer ...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
650,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"['Kevin Krewell', 'Amy Danise', 'Contributor G...",2022-11-17 00:00:00,IBM has been committed to quantum computing be...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...


In [20]:
quantum_df

,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body
0,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,Imagine a world where the most widely-used cry...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
1,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,If you take the development of serious quantum...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
3,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,"All is not doom and gloom, however. There are ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
4,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,"Strong symmetric ciphers, algorithms that use ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
5,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,But let’s just reiterate that the hollowing-ou...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...
...,...,...,...,...,...,...,...,...,...,...
668,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,With the development of quantum computing tech...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
669,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,IBM Fellow and VP of Quantum Jay Gambetta ende...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
670,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,Even though the concept of a quantum computer ...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...
671,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,IBM has been committed to quantum computing be...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...


In [3]:
import nlp_analysis.word_wizard as ww
wizard = ww.WordWizard(df=quantum_df)
wizard.create_word_embeddings(columns=["body"], lean=True) \
      .cluster_embeddings("body")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating embeddings for column(s): ['body']: 100%|██████████| 1/1 [01:26<00:00, 86.30s/it]


Optimal K: 14


In [21]:
wizard.df

,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body,sentences,body_word_embeddings,body_word_embeddings_cluster_13,body_word_embeddings_cluster_13_is_medoid,body_word_embeddings_cluster_14,body_word_embeddings_cluster_14_is_medoid,body_word_embeddings_cluster_15,body_word_embeddings_cluster_15_is_medoid
0,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,Imagine a world where the most widely-used cry...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,[Imagine a world where the most widely-used cr...,"[0.35911059379577637, -0.026911400258541107, 0...",3,True,1,True,1,True
1,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,If you take the development of serious quantum...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,[If you take the development of serious quantu...,"[0.35911059379577637, -0.026911400258541107, 0...",3,True,1,True,1,True
2,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,"All is not doom and gloom, however. There are ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,"[All is not doom and gloom, however., There ar...","[0.35911059379577637, -0.026911400258541107, 0...",3,True,1,True,1,True
3,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,"Strong symmetric ciphers, algorithms that use ...",Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,"[Strong symmetric ciphers, algorithms that use...","[0.35911059379577637, -0.026911400258541107, 0...",3,True,1,True,1,True
4,0,Bing,https://hackaday.com/2015/09/29/quantum-comput...,Imagine a world where the most widely-used cry...,Hackaday,[Elliot Williams],2015-09-29 00:00:00,But let’s just reiterate that the hollowing-ou...,Quantum Computing Kills Encryption,Imagine a world where the most widely-used cry...,[But let’s just reiterate that the hollowing-o...,"[0.35911059379577637, -0.026911400258541107, 0...",3,True,1,True,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,With the development of quantum computing tech...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...,[With the development of quantum computing tec...,"[0.3131255805492401, -0.1540013551712036, 0.03...",12,False,8,False,7,False
648,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,IBM Fellow and VP of Quantum Jay Gambetta ende...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...,[IBM Fellow and VP of Quantum Jay Gambetta end...,"[0.3131255805492401, -0.1540013551712036, 0.03...",12,False,8,False,7,False
649,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit the company revea...,Forbes,"[Kevin Krewell, Amy Danise, Contributor Group]",2022-11-17 00:00:00,Even though the concept of a quantum computer ...,IBM Prepares For Quantum Computing Inflection ...,IBM Quantum System 2 artist concept IBM\n\nIBM...,[Even though the concept of a quantum computer...,"[0.3131255805492401, -0.1540013551712036, 0.03...",12,False,8,False,7,False
650,49,Google,https://www.forbes.com/sites/tiriasresearch/20...,At IBM's 2022 Quantum Summit

In [15]:
# grouping wizard.df by body_word_embeddings_cluster_14 and counting medoids in each cluster
wizard.df.groupby("body_word_embeddings_cluster_13").agg({"body_word_embeddings_cluster_13_is_medoid": "sum"})

,body_word_embeddings_cluster_13_is_medoid
body_word_embeddings_cluster_13,
0,0
1,0
2,0
3,13
4,0
5,0
6,0
7,0
8,0


In [18]:
wizard.df[wizard.df['body_word_embeddings_cluster_15_is_medoid'] == True]['body_word_embeddings']

0     [0.35911059379577637, -0.026911400258541107, 0...
1     [0.35911059379577637, -0.026911400258541107, 0...
2     [0.35911059379577637, -0.026911400258541107, 0...
3     [0.35911059379577637, -0.026911400258541107, 0...
4     [0.35911059379577637, -0.026911400258541107, 0...
5     [0.35911059379577637, -0.026911400258541107, 0...
6     [0.35911059379577637, -0.026911400258541107, 0...
7     [0.35911059379577637, -0.026911400258541107, 0...
8     [0.35911059379577637, -0.026911400258541107, 0...
9     [0.35911059379577637, -0.026911400258541107, 0...
10    [0.35911059379577637, -0.026911400258541107, 0...
11    [0.35911059379577637, -0.026911400258541107, 0...
12    [0.35911059379577637, -0.026911400258541107, 0...
13    [0.35911059379577637, -0.026911400258541107, 0...
14    [0.35911059379577637, -0.026911400258541107, 0...
Name: body_word_embeddings, dtype: object

In [19]:
wizard.df['body_word_embeddings']

0      [0.35911059379577637, -0.026911400258541107, 0...
1      [0.35911059379577637, -0.026911400258541107, 0...
2      [0.35911059379577637, -0.026911400258541107, 0...
3      [0.35911059379577637, -0.026911400258541107, 0...
4      [0.35911059379577637, -0.026911400258541107, 0...
                             ...                        
647    [0.3131255805492401, -0.1540013551712036, 0.03...
648    [0.3131255805492401, -0.1540013551712036, 0.03...
649    [0.3131255805492401, -0.1540013551712036, 0.03...
650    [0.3131255805492401, -0.1540013551712036, 0.03...
651    [0.3131255805492401, -0.1540013551712036, 0.03...
Name: body_word_embeddings, Length: 652, dtype: object